In [1]:
pip install rdflib

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install wget

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9680 sha256=09fb3900ade65e0d82534e3f93fd324815b976d29007150a019ee3249de39989
  Stored in directory: c:\users\teunh\appdata\local\pip\cache\wheels\40\b3\0f\a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget
Note: you may need to restart the kernel to use updated packages.


In [6]:
import wget

url = "https://dgl-data.s3-us-west-2.amazonaws.com/dataset/DRKG/drkg.tar.gz"
DRKG_data = wget.download(url)
 

100% [......................................................................] 216650245 / 216650245

In [10]:
import os
import tarfile

def download_and_extract():
    import shutil
    import requests
    
    url = "https://s3.us-west-2.amazonaws.com/dgl-data/dataset/DRKG/drkg.tar.gz"
    path = "../data/"
    filename = "drkg.tar.gz"
    fn = os.path.join(path, filename)
    if os.path.exists("../data/drkg/drkg.tsv"):
        return
    
    opener, mode = tarfile.open, 'r:gz'
    os.makedirs(path, exist_ok=True)
    cwd = os.getcwd()
    os.chdir(path)
    while True:
        try:
            file = opener(filename, mode)
            try: file.extractall()
            finally: file.close()
            break
        except Exception:
            f_remote = requests.get(url, stream=True)
            sz = f_remote.headers.get('content-length')
            assert f_remote.status_code == 200, 'fail to open {}'.format(url)
            with open(filename, 'wb') as writer:
                for chunk in f_remote.iter_content(chunk_size=1024*1024):
                    writer.write(chunk)
            print('Download finished. Unzipping the file...')
    os.chdir(cwd)

In [1]:
import pandas as pd
import numpy as np


drkg_file = './drkg.tsv'
df = pd.read_csv(drkg_file, sep ="\t", header=None)
triplets = df.values.tolist()

In [2]:
triplets[200000:210000]

[['Compound::DB00206',
  'DRUGBANK::ddi-interactor-in::Compound:Compound',
  'Compound::DB00598'],
 ['Compound::DB00206',
  'DRUGBANK::ddi-interactor-in::Compound:Compound',
  'Compound::DB00866'],
 ['Compound::DB00206',
  'DRUGBANK::ddi-interactor-in::Compound:Compound',
  'Compound::DB00960'],
 ['Compound::DB00206',
  'DRUGBANK::ddi-interactor-in::Compound:Compound',
  'Compound::DB01182'],
 ['Compound::DB00206',
  'DRUGBANK::ddi-interactor-in::Compound:Compound',
  'Compound::DB01193'],
 ['Compound::DB00206',
  'DRUGBANK::ddi-interactor-in::Compound:Compound',
  'Compound::DB01295'],
 ['Compound::DB00206',
  'DRUGBANK::ddi-interactor-in::Compound:Compound',
  'Compound::DB01297'],
 ['Compound::DB00206',
  'DRUGBANK::ddi-interactor-in::Compound:Compound',
  'Compound::DB01359'],
 ['Compound::DB00206',
  'DRUGBANK::ddi-interactor-in::Compound:Compound',
  'Compound::DB01580'],
 ['Compound::DB00206',
  'DRUGBANK::ddi-interactor-in::Compound:Compound',
  'Compound::DB03322'],
 ['Compoun

In [3]:
from rdflib import Graph, URIRef, Literal, Namespace, RDF, RDFS

#keep track of all unique entities with the respective URI
uri_data = [{"entity_type", "uri"}]

# Initialize an RDF graph
g = Graph()

SCHEMA = Namespace("https://schema.org/")
CKO = Namespace("https://w3id.org/climatekg/ontology/")
EX = Namespace("http://example.org/")
rdf = RDF
rdfs = RDFS

#bind prefixes
g.bind("schema", SCHEMA)
g.bind("cko", CKO)
g.bind("ex", EX)

#now add all the unique entities to the graph
all_unique_entities = set()
for triplet in triplets:
    src, _, dest = triplet
    all_unique_entities.update([src, dest])

uri_data = []  # Use a list to store dictionaries

for unique_entity in all_unique_entities:
    splitted = unique_entity.split("::")
    node_type = splitted[0].replace(' ', '_')
    entity = splitted[1].replace(' ', '_').replace('|', '_')
    uri = URIRef(f"https://schema.org/{node_type}/{entity}")
    
    g.add((uri, RDF.type, SCHEMA[node_type]))
    g.add((uri, RDFS.label, Literal(unique_entity)))
    # Append as a dictionary with explicit keys
    uri_data.append({'Entity': unique_entity, 'URI': str(uri)})

# Create a DataFrame with column names
uri_df = pd.DataFrame(uri_data, columns=['Entity', 'URI'])
print(uri_df[5:10])

                           Entity  \
5               Compound::DB09076   
6                    Atc::L02BG02   
7               Compound::DB15172   
8  Biological Process::GO:0046578   
9  Biological Process::GO:1900103   

                                                URI  
5               https://schema.org/Compound/DB09076  
6                    https://schema.org/Atc/L02BG02  
7               https://schema.org/Compound/DB15172  
8  https://schema.org/Biological_Process/GO:0046578  
9  https://schema.org/Biological_Process/GO:1900103  


In [18]:
def generate_relation_uri(relation_type):
    # This is a simple example. Adjust the logic to fit your dataset and naming conventions.
    relation = relation_type.split("::")
    relation = relation[-1].split(':')[0]# Simplify or adjust as needed
    base_url = "http://example.org/relation/"
    relation_uri = URIRef(f"{base_url}{relation.replace(' ', '_')}")
    return relation_uri

In [19]:
#allright now we have every entity saved to a URI, both in a df and in the graph, lets form the triples
for src, relation_type, dest in triplets:
    # Assuming 'uri_df' is your DataFrame with a mapping from entity identifiers to URIs
    # And 'Entity' is the column with the entity identifier, 'URI' is the column with the URI
    src_uri = uri_df.loc[uri_df['Entity'] == src, 'URI'].values[0]
    dest_uri = uri_df.loc[uri_df['Entity'] == dest, 'URI'].values[0]
    relation_uri = generate_relation_uri(relation_type)

    print("src: " + src_uri)
    print("dest: " + dest_uri)
    print("relation: " + relation_uri)

    # Add the relationship to the graph
    g.add((src_uri_ref, relation_uri, dest_uri_ref))

relation: http://example.org/relation/HumGenHumGen does not look like a valid URI, trying to serialize this will break.
relation: http://example.org/relation/HumGenHumGen does not look like a valid URI, trying to serialize this will break.
relation: http://example.org/relation/HumGenHumGen does not look like a valid URI, trying to serialize this will break.
relation: http://example.org/relation/HumGenHumGen does not look like a valid URI, trying to serialize this will break.
relation: http://example.org/relation/HumGenHumGen does not look like a valid URI, trying to serialize this will break.
relation: http://example.org/relation/HumGenHumGen does not look like a valid URI, trying to serialize this will break.


src: https://schema.org/Gene/2157
dest: https://schema.org/Gene/2157
relation: http://example.org/relation/HumGenHumGen
src: https://schema.org/Gene/2157
dest: https://schema.org/Gene/5264
relation: http://example.org/relation/HumGenHumGen
src: https://schema.org/Gene/2157
dest: https://schema.org/Gene/2158
relation: http://example.org/relation/HumGenHumGen
src: https://schema.org/Gene/2157
dest: https://schema.org/Gene/3309
relation: http://example.org/relation/HumGenHumGen
src: https://schema.org/Gene/2157
dest: https://schema.org/Gene/28912
relation: http://example.org/relation/HumGenHumGen
src: https://schema.org/Gene/2157
dest: https://schema.org/Gene/811
relation: http://example.org/relation/HumGenHumGen


relation: http://example.org/relation/HumGenHumGen does not look like a valid URI, trying to serialize this will break.
relation: http://example.org/relation/HumGenHumGen does not look like a valid URI, trying to serialize this will break.
relation: http://example.org/relation/HumGenHumGen does not look like a valid URI, trying to serialize this will break.
relation: http://example.org/relation/HumGenHumGen does not look like a valid URI, trying to serialize this will break.
relation: http://example.org/relation/HumGenHumGen does not look like a valid URI, trying to serialize this will break.
relation: http://example.org/relation/HumGenHumGen does not look like a valid URI, trying to serialize this will break.
relation: http://example.org/relation/HumGenHumGen does not look like a valid URI, trying to serialize this will break.


src: https://schema.org/Gene/2157
dest: https://schema.org/Gene/2159
relation: http://example.org/relation/HumGenHumGen
src: https://schema.org/Gene/2157
dest: https://schema.org/Gene/821
relation: http://example.org/relation/HumGenHumGen
src: https://schema.org/Gene/2157
dest: https://schema.org/Gene/5627
relation: http://example.org/relation/HumGenHumGen
src: https://schema.org/Gene/2157
dest: https://schema.org/Gene/5624
relation: http://example.org/relation/HumGenHumGen
src: https://schema.org/Gene/2157
dest: https://schema.org/Gene/2147
relation: http://example.org/relation/HumGenHumGen
src: https://schema.org/Gene/5141
dest: https://schema.org/Gene/5141
relation: http://example.org/relation/HumGenHumGen
src: https://schema.org/Gene/5141
dest: https://schema.org/Gene/4067


relation: http://example.org/relation/HumGenHumGen does not look like a valid URI, trying to serialize this will break.
relation: http://example.org/relation/HumGenHumGen does not look like a valid URI, trying to serialize this will break.
relation: http://example.org/relation/HumGenHumGen does not look like a valid URI, trying to serialize this will break.
relation: http://example.org/relation/HumGenHumGen does not look like a valid URI, trying to serialize this will break.
relation: http://example.org/relation/HumGenHumGen does not look like a valid URI, trying to serialize this will break.
relation: http://example.org/relation/HumGenHumGen does not look like a valid URI, trying to serialize this will break.
relation: http://example.org/relation/HumGenHumGen does not look like a valid URI, trying to serialize this will break.


...

relation: http://example.org/relation/HumGenHumGen does not look like a valid URI, trying to serialize this will break.
relation: http://example.org/relation/HumGenHumGen does not look like a valid URI, trying to serialize this will break.
relation: http://example.org/relation/HumGenHumGen does not look like a valid URI, trying to serialize this will break.
relation: http://example.org/relation/HumGenHumGen does not look like a valid URI, trying to serialize this will break.
relation: http://example.org/relation/HumGenHumGen does not look like a valid URI, trying to serialize this will break.
relation: http://example.org/relation/HumGenHumGen does not look like a valid URI, trying to serialize this will break.


src: https://schema.org/Gene/3845
dest: https://schema.org/Gene/3692
relation: http://example.org/relation/HumGenHumGen
src: https://schema.org/Gene/3845
dest: https://schema.org/Gene/5516
relation: http://example.org/relation/HumGenHumGen
src: https://schema.org/Gene/3845
dest: https://schema.org/Gene/983
relation: http://example.org/relation/HumGenHumGen
src: https://schema.org/Gene/3845
dest: https://schema.org/Gene/3181
relation: http://example.org/relation/HumGenHumGen
src: https://schema.org/Gene/3845
dest: https://schema.org/Gene/1497
relation: http://example.org/relation/HumGenHumGen
src: https://schema.org/Gene/3845
dest: https://schema.org/Gene/81502
relation: http://example.org/relation/HumGenHumGen


relation: http://example.org/relation/HumGenHumGen does not look like a valid URI, trying to serialize this will break.
relation: http://example.org/relation/HumGenHumGen does not look like a valid URI, trying to serialize this will break.
relation: http://example.org/relation/HumGenHumGen does not look like a valid URI, trying to serialize this will break.
relation: http://example.org/relation/HumGenHumGen does not look like a valid URI, trying to serialize this will break.
relation: http://example.org/relation/HumGenHumGen does not look like a valid URI, trying to serialize this will break.
relation: http://example.org/relation/HumGenHumGen does not look like a valid URI, trying to serialize this will break.


src: https://schema.org/Gene/3845
dest: https://schema.org/Gene/6286
relation: http://example.org/relation/HumGenHumGen
src: https://schema.org/Gene/3845
dest: https://schema.org/Gene/6541
relation: http://example.org/relation/HumGenHumGen
src: https://schema.org/Gene/3845
dest: https://schema.org/Gene/8833
relation: http://example.org/relation/HumGenHumGen
src: https://schema.org/Gene/3845
dest: https://schema.org/Gene/2935
relation: http://example.org/relation/HumGenHumGen
src: https://schema.org/Gene/3845
dest: https://schema.org/Gene/9475
relation: http://example.org/relation/HumGenHumGen
src: https://schema.org/Gene/3845
dest: https://schema.org/Gene/6574
relation: http://example.org/relation/HumGenHumGen


KeyboardInterrupt: 

In [ ]:
#serialize graph
g.serialize(destination="my_knowledge_graph.ttl", format='turtle')

In [29]:
all_unique_entities

{'Gene::20821',
 'Biological Process::GO:0048769',
 'Compound::CHEMBL3287735',
 'Pathway::PC7_3663',
 'Gene::1889',
 'Gene::drugbank:BE0001513',
 'Compound::brenda:25412',
 'Compound::DB11751',
 'Gene::5550',
 'Gene::6833',
 'Disease::MESH:C537038',
 'Disease::MESH:C562830',
 'Gene::22876',
 'Gene::323299',
 'Biological Process::GO:0070235',
 'Gene::399162',
 'Gene::63920',
 'Gene::1573',
 'Biological Process::GO:0098657',
 'Gene::8739',
 'Gene::100040591',
 'Gene::440270',
 'Gene::3416',
 'Gene::59344',
 'Side Effect::C0152202',
 'Biological Process::GO:0048496',
 'Gene::83429',
 'Side Effect::C0221161',
 'Cellular Component::GO:0070469',
 'Compound::CHEMBL3989911',
 'Gene::94240',
 'Biological Process::GO:0031508',
 'Gene::327712',
 'Disease::MESH:D007570',
 'Atc::M04AX',
 'Gene::2703434',
 'Molecular Function::GO:0045145',
 'Compound::DB06153',
 'Compound::MESH:D008753',
 'Symptom::D000472',
 'Molecular Function::GO:0016502',
 'Compound::DB07347',
 'Compound::DB04863',
 'Biological 